# Day 15: Chiton
This has to be a problem for 
[Dijkstra's Algorithm](https://en.wikipedia.org/wiki/Dijkstra%27s_algorithm)
([Computerphile's breakdown](https://www.youtube.com/watch?v=GazC3A4OQTE))
but Dijkstra doesn't account for weights per point it just does shortes path.

A* does account for weights
([Computerphile's breakdown](https://www.youtube.com/watch?v=ySN5Wnu88nE))

In [ ]:
#r "nuget: astar-search"
open AStar

let int (ch:char) :int = int ch - int '0'
let sample = "1163751742
1381373672
2136511328
3694931569
7463417111
1319128137
1359912421
3125421639
1293138521
2311944581"

let grid = sample.Split(Environment.NewLine) |> Array.map (fun s -> Seq.map float s |> Seq.toArray)
let width,height = Array.length (Array.head grid), Array.length grid
let start = 0,0
let goal = width - 1, height - 1
let neighbours (x, y) =
    let found = 
        [-1..1] |> List.collect (fun nx ->
            [-1..1] |> List.filter (fun ny -> 
                // ny <> nx || ny <> 0
                (nx = x && ny <> y) || (nx <> x && ny = y)
            ) 
            |> List.map (fun ny -> 
                x + nx, y + ny
            )
        )
    found |> Seq.filter (fun (nx, ny) -> 
        nx >= 0 && ny >= 0 &&
        nx < width && ny < height)
let gScore _ (x,y) =
    grid |> Array.item y |> Array.item x
let fScore (x, y) (gx, gy) = sqrt ((float gx - float x)**2. + (float gy - float y)**2.)

match search start goal { neighbours = neighbours; gCost = gScore; fCost = fScore; maxIterations = None } with
| Some path ->
    path |> Seq.rev |> display |> ignore
    // grid |> Array.mapi (fun y row -> 
    //     row 
    //     |> Array.mapi (fun x cell -> if Seq.contains (x, y) path then "#" else cell.ToString())
    //     |> String.concat ""
    // ) |> Array.iter (printfn "%s")
| None -> printf "No path"

Installed Packages astar-search, 1.0.2

No path